# Training
> training loop

In [ ]:
#| default_exp training

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
from slg_generative.models.vae import AutoEncoder
from slg_generative.data.datasets import FashionMnistDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
import torch

In [ ]:
#| export

class Trainer:
    "Trainer for VAE models"

    def __init__(self,
        model:AutoEncoder, # Model
        dataloader:torch.utils.data.DataLoader, # Dataloader
        loss_func:torch.nn.modules.loss._Loss, # Loss function
        optimizer:torch.optim.Optimizer, # Optimizer
        n_epochs:int, # Number of training epochs
        device:str # Device
    ):
        self.model = model
        self.dataloader = dataloader
        self.loss_func = loss_func
        self.optimizer = optimizer
        self.n_epochs = n_epochs
        self.device = device

    def fit(self):
        # training loop
        for epoch in tqdm(range(self.n_epochs)):
            for batch_idx, (x,y) in enumerate(self.dataloader):
                x = x.to(self.device)
                self.optimizer.zero_grad()
                x_hat = self.model(x)
                loss = self.loss_func(x_hat, x)
                loss.backward()
                self.optimizer.step()
                if batch_idx % 100:
                    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch+1,
                        self.n_epochs,
                        batch_idx * len(x), 
                        len(self.dataloader.dataset),
                        100. * batch_idx / len(self.dataloader), 
                        loss.cpu().data.item()), 
                        end='')


In [ ]:
# device
# device = 'mps' if torch.backends.mps.is_available() else 'cpu' # or 'cuda' for nvidia gpus
device = 'cpu'
# data
ds = FashionMnistDataset(csv_file="~/Data/fashion-mnist/fashion-mnist_train.csv")
dl = torch.utils.data.DataLoader(ds,batch_size=128,shuffle=True)
# model
autoencoder = AutoEncoder().to(device)
# training params
n_epochs = 5
# optim
opt = Adam(autoencoder.parameters(), lr=1e-3)
# # mean square error loss 
loss_func = nn.MSELoss()

In [ ]:
trainer = Trainer(autoencoder, dl, loss_func, opt, n_epochs, device)
trainer.fit()

  0%|          | 0/5 [00:00<?, ?it/s]

 Train Epoch: 1/5 [59776/60000 (100%)]	Loss: 0.634618

 20%|██        | 1/5 [00:04<00:19,  4.86s/it]

 Train Epoch: 2/5 [56960/60000 (95%)]	Loss: 0.6342823

 40%|████      | 2/5 [00:07<00:10,  3.64s/it]

...

 Train Epoch: 4/5 [59008/60000 (98%)]	Loss: 0.6307802

 80%|████████  | 4/5 [00:13<00:03,  3.08s/it]

 Train Epoch: 5/5 [57216/60000 (95%)]	Loss: 0.6383358

100%|██████████| 5/5 [00:16<00:00,  3.20s/it]

 Train Epoch: 5/5 [44928/60000 (100%)]	Loss: 0.604993

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()